<a href="https://colab.research.google.com/github/adarsh-nl/-adarshnl.github.io./blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [ ]:
from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
import cv2

from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
import imutils

# Initialize directory path variable 

In [ ]:
store_image=[]
train_path=r'.\train'
fps=5
train_videos='/content/drive/MyDrive/train/training_videos'
train_images_path='/content/frames'
#os.makedirs(train_images_path)



# function to process and store video frames

In [ ]:
def store_inarray(image_path):
    image=load_img(image_path)
    image=img_to_array(image)
    image=cv2.resize(image, (227,227), interpolation = cv2.INTER_AREA)
    gray=0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)

# Extract frames from videos 
PS: Dont run unwantedly

In [ ]:
for video in train_videos:
    os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(train_path,video,fps,train_path))
    images=os.listdir(train_images_path)
    for image in images:
        image_path=train_image_path + '/' + image
        store_inarray(image_path)

In [ ]:
# Importing all necessary libraries 
import cv2 
import os 

# Read the video from specified path 
cam = cv2.VideoCapture(r"C:\Users\adars\OneDrive\Desktop\Video surveillance\train\training_videos\16.avi")

try: 
	
	# creating a folder named data 
	if not os.path.exists('data'): 
		os.makedirs('data') 

# if not created then raise error 
except OSError: 
	print ('Error: Creating directory of data') 

# frame 
currentframe = 15086

while(True): 
	
	# reading from frame 
	ret,frame = cam.read() 

	if ret: 
		# if video is still left continue creating images 
		name = './data/frame' + str(currentframe) + '.jpg'
		print ('Creating...' + name) 

		# writing the extracted images 
		cv2.imwrite(name, frame) 

		# increasing counter so that it will 
		# show how many frames are created 
		currentframe += 1
	else: 
		break

# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows() 


# Store the images in an array 

using the above mentioned function 'store_inarray'

In [ ]:
images=os.listdir(train_images_path)
for image in images:
    image_path=train_images_path + '/' + image
    store_inarray(image_path)

In [ ]:
#print(store_image.shape)

# Store the store_image list in a numpy file “training.npy”

In [ ]:
store_image=np.array(store_image)

#print(store_image.shape)
a,b,c=store_image.shape

store_image.resize(b,c,a)
store_image=(store_image-store_image.mean())/(store_image.std())
store_image=np.clip(store_image,0,1)
np.save('training.npy',store_image)

# Creating auto encoder architecture

In [ ]:
stae_model=Sequential()

stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [ ]:
training_data=np.load('training.npy')
frames=training_data.shape[2]
frames=frames-frames%10

In [ ]:
training_data=training_data[:,:,:frames]
training_data=training_data.reshape(-1,227,227,10)
training_data=np.expand_dims(training_data,axis=4)
target_data=training_data.copy()

# Creating Check point 

In [ ]:
checkpoint_path=r"/content/drive/MyDrive/train/cp.ckpt"

In [ ]:
checkpoint_dir = os.path.dirname(checkpoint_path)

#create a call back that saves the models weights
cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

In [ ]:
epochs=10
batch_size=1

In [ ]:
stae_model.fit(training_data,target_data, batch_size=batch_size, epochs=epochs, callbacks = [cp_callback])

Epoch 1/10
1436/1436 [==============================] - 3298s 2s/step - loss: 0.1098 - accuracy: 0.6847

Epoch 00001: saving model to /content/drive/MyDrive/train/cp.ckpt
Epoch 2/10
1436/1436 [==============================] - 3293s 2s/step - loss: 0.0692 - accuracy: 0.7378

Epoch 00002: saving model to /content/drive/MyDrive/train/cp.ckpt
Epoch 3/10
1436/1436 [==============================] - 3256s 2s/step - loss: 0.0685 - accuracy: 0.7383

Epoch 00003: saving model to /content/drive/MyDrive/train/cp.ckpt
Epoch 4/10
1436/1436 [==============================] - 3237s 2s/step - loss: 0.0487 - accuracy: 0.7541

Epoch 00004: saving model to /content/drive/MyDrive/train/cp.ckpt
Epoch 5/10
1436/1436 [==============================] - 3330s 2s/step - loss: 0.0316 - accuracy: 0.7725

Epoch 00005: saving model to /content/drive/MyDrive/train/cp.ckpt
Epoch 6/10
1436/1436 [==============================] - 3335s 2s/step - loss: 0.0277 - accuracy: 0.7766

Epoch 00006: saving model to /content/dr

In [ ]:
stae_model.save('my_model.h5')

# **Mounting Drive to access the files from drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/train"

checkpoint		     cp.ckpt.index  training_videos
cp.ckpt.data-00000-of-00001  frames.zip


In [ ]:
!unzip -q "/content/drive/MyDrive/train/frames.zip"